<a href="https://colab.research.google.com/github/ItalianPepper/Autoencoder_gan/blob/master/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import os.path
import numpy as np
import cv2 as cv
import random
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import *

#https://towardsdatascience.com/10-lessons-i-learned-training-generative-adversarial-networks-gans-for-a-year-c9071159628


def get_dataset(path, img_shape):
    paths = []

    valid_format = [".jpg", ".png", ".tiff", ".bmp", ".gif"]

    for img in os.listdir(path):

        extension = os.path.splitext(img)[1]

        if extension.lower() in valid_format:
            paths.append(os.path.join(path, img))

    k_train = []

    for path_image in paths:
        img = load_img(path_image, target_size=img_shape[:2])
        
        el = img_to_array(img) / 255.0
        
        k_train.append(el)

    k_train = np.asarray(k_train)

    return k_train

  
def get_noise(n_sample=1, nlatent_dim=512):
    noise = np.random.normal(0, 1, (n_sample, nlatent_dim))
    return (noise)
  

def plot_generated_images(noise, nsample=1, path_save=None, epoch=0):
    imgs = decoder.predict(noise)
    img = imgs[0]
    
    # Solution: "Clipping input data to the valid range for imshow 
    # with RGB data ([0..1] for floats or [0..255] for integers)."
    
    img = (img * 255).astype(np.uint8)
    
    fig = plt.figure(figsize=(40, 10))
    
    epoch_str = str(epoch)
    extension = ".png"
    path_name = path_save + "/" + epoch_str + " " + extension
    
    ax = fig.add_subplot(1, nsample, 1)
    ax.imshow(img)
    
    fig.suptitle("Epoch:" + epoch_str, fontsize=30)

    plt.savefig(path_name,
                bbox_inches='tight',
                pad_inches=0)
    plt.close()

def build_encoder(img_shape):
    input_img = Input(shape=img_shape)
    n_filter = 32
    
    # Encoding
    x = Conv2D(n_filter, kernel_size=(3, 3), padding="same")(input_img)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(n_filter*8, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Flatten()(x)
    #mean = Dense(512)(x)
    logsigma = Dense(512, activation="tanh")(x)
    #latent = Lambda(sampling, output_shape=(512,))([mean, logsigma])
    
    model = Model(inputs=input_img, outputs=logsigma)
    model.summary()
    return (model)
  
def build_decoder_gen(en_shape=(512,)):
    # impostazione del generatore
    en_img = Input(shape=en_shape)
    n_filter = 32
    
    x = Dense(512*8*8)(en_img)
    x = Reshape((8,8,512))(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # Decoding
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(n_filter*2, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(n_filter, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(3, kernel_size=(3, 3), padding="same")(x)
    x = Activation('tanh')(x)
    
    model = Model(inputs=en_img, outputs=x)
    model.summary()
    return (model)
  
def build_discriminator(img_shape):
  
    input_img = Input(shape=img_shape)
    n_filter = 32
    
    x = Conv2D(n_filter, kernel_size=(3, 3), padding='same', strides=2)(input_img)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(n_filter*2, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, kernel_size=(3, 3), padding='same', strides=2)(x)
    
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)

    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=out)

    model.summary()
    return model
  
  
def train(encoder, decoder, discriminator, gan, x_train,
          dir_result="./", epochs=10000, batch_size=512):
   
    history = []
    
    if epochs >= 100:
      checkpoint = int(epochs/100)
    elif epochs < 100:
      checkpoint = 1
    
    
    for epoch in range(epochs):
      
        start_epoch = time.time()
        
        np.random.shuffle(x_train)
        
        batches_index = [x for x in range(0, len(x_train)+1, batch_size)]
        
        if batches_index[len(batches_index)-1] != len(x_train):
          batches_index.append(len(x_train))
        #print(batches_index)
        
        for i in range(0, len(batches_index)-1):
          
          start = batches_index[i]
          
          # if i == len(batches_index)-1:
          #   end = len(x_train)
          # else:
          end = batches_index[i+1]
          #print("Batch ",str(i),":",start, end)  
          
          x_batch = x_train[start:end]
          x_batch_size = len(x_batch)
          
          
          # encoder e decoder su immagini reali
          gen_img_enc = encoder.predict(x_batch)
          gen_img_dec = decoder.predict(gen_img_enc)

          noise = get_noise(n_sample=x_batch_size)
          # immagini create dal noise
          fake_img = decoder.predict(noise)

          matrix_true = np.ones((x_batch_size, 1))
          matrix_false = np.zeros((x_batch_size, 1))
          
          # Allenamento discriminatore
          dsc_true = discriminator.train_on_batch(x_batch, matrix_true)
          dsc_true_enc = discriminator.train_on_batch(gen_img_dec, matrix_true)
          dsc_fake = discriminator.train_on_batch(fake_img, matrix_false)

          # Allenamento decoder
          
          noise_gen = get_noise(n_sample=x_batch_size)
          decoder_noise_loss = decoder.train_on_batch(noise_gen, x_batch)
          
          gan_loss = gan.train_on_batch(x_batch, matrix_true)
          
        if epoch % checkpoint == 0 and epoch > 0:
          
          history.append({"Epoch:": epoch,
                          "Discriminator_loss_true:": dsc_true[0],
                          "Discriminator_loss_true_enc:": dsc_true_enc[0],
                          "Discriminator_loss_fake": dsc_fake[0],
                          "Discriminator_acc_fake": dsc_fake[1],
                          "Decoder_noise_loss": decoder_noise_loss,
                          "GAN_Loss:": gan_loss})
          
          print({"Epoch:": epoch,
                          "Discriminator_loss_true:": dsc_true[0],
                          "Discriminator_loss_true_enc:": dsc_true_enc[0],
                          "Discriminator_loss_fake": dsc_fake[0],
                          "Discriminator_acc_fake": dsc_fake[1],
                          "Decoder_noise_loss": decoder_noise_loss,
                          "GAN_Loss:": gan_loss})
          
          noise = get_noise(n_sample=1)

          plot_generated_images(noise, 1, dir_result, epoch)
          
        end_epoch = time.time()
        end_epoch = end_epoch - start_epoch
        print("Ended Epoch {:0.0f}/{:1.0f} in: {:2.4f} s".format(epoch + 1, epochs, end_epoch))

    return history

start = time.time()
   
img_shape = (64, 64, 3)

optimizer_std = Adam(0.0001)

encoder = build_encoder(img_shape)
decoder = build_decoder_gen()
discriminator = build_discriminator(img_shape)

encoder.compile(loss="mse", optimizer=optimizer_std)
decoder.compile(loss="mse", optimizer=optimizer_std)

discriminator.trainable = False

discriminator.compile(loss="mse",
                              optimizer=optimizer_std,
                              metrics=["accuracy"])

x = Input(shape=img_shape)
z = encoder(x)
out_dec = decoder(z)
out = discriminator(out_dec)

gan = Model(inputs=x, outputs=out)

gan.compile(loss="mse", optimizer=optimizer_std)

x_train = get_dataset("./img_align_celeba", img_shape)

path = "./drive/My Drive/GAN/res_gan_autoencoder/"

history = train(encoder, decoder, discriminator, gan, x_train,
                  dir_result=path, epochs=1000, batch_size=1000)

with open(path+"log_gan.txt", "w") as f:
  for item in history:
    item_str = str(item) + "\n"
    f.write(item_str)

# Altrimenti i pesi non verranno salvati.
discriminator.trainable = True

encoder.save(path+"encoder.h5")
decoder.save(path+"decoder.h5")
discriminator.save(path+"discriminator.h5")
# La gan va ricreata a causa di: https://github.com/keras-team/keras/issues/9589
gan.save(path+"gan_autoencoder.h5")

end = time.time()
end = end - start
print("Finished in: " + str(end) + " s")

In [0]:
!unzip -Z1 "/content/drive/My Drive/GAN/img_align_celeba.zip" | head -5001 | sed 's| |\\ |g' | xargs unzip "/content/drive/My Drive/GAN/img_align_celeba.zip" -d ./

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rm -r img_align_celeba

In [0]:
#128, 128, 3

import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import os.path
import numpy as np
import cv2 as cv
import random
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import *

#https://towardsdatascience.com/10-lessons-i-learned-training-generative-adversarial-networks-gans-for-a-year-c9071159628


def get_dataset(path, img_shape):
    paths = []

    valid_format = [".jpg", ".png", ".tiff", ".bmp", ".gif"]

    for img in os.listdir(path):

        extension = os.path.splitext(img)[1]

        if extension.lower() in valid_format:
            paths.append(os.path.join(path, img))

    k_train = []

    for path_image in paths:
        img = load_img(path_image, target_size=img_shape[:2])
        
        el = img_to_array(img) / 255.0
        
        k_train.append(el)

    k_train = np.asarray(k_train)

    return k_train

  
def get_noise(n_sample=1, nlatent_dim=512):
    noise = np.random.normal(0, 1, (n_sample, nlatent_dim))
    return (noise)
  

def plot_generated_images(noise, nsample=1, path_save=None, epoch=0):
    imgs = decoder.predict(noise)
    img = imgs[0]
    
    # Solution: "Clipping input data to the valid range for imshow 
    # with RGB data ([0..1] for floats or [0..255] for integers)."
    
    img = (img * 255).astype(np.uint8)
    
    fig = plt.figure(figsize=(40, 10))
    
    epoch_str = str(epoch)
    extension = ".png"
    path_name = path_save + "/" + epoch_str + " " + extension
    
    ax = fig.add_subplot(1, nsample, 1)
    ax.imshow(img)
    
    fig.suptitle("Epoch:" + epoch_str, fontsize=30)

    plt.savefig(path_name,
                bbox_inches='tight',
                pad_inches=0)
    plt.close()

def build_encoder(img_shape):
    input_img = Input(shape=img_shape)
    n_filter = 32
    
    # Encoding
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding="same")(input_img)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(n_filter*8, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(n_filter*16, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Flatten()(x)
    #mean = Dense(512)(x)
    logsigma = Dense(512, activation="tanh")(x)
    #latent = Lambda(sampling, output_shape=(512,))([mean, logsigma])
    
    model = Model(inputs=input_img, outputs=logsigma)
    model.summary()
    return (model)
  
def build_decoder_gen(en_shape=(512,)):
    # impostazione del generatore
    en_img = Input(shape=en_shape)
    n_filter = 32
    
    x = Dense(512*16*16)(en_img)
    x = Reshape((16,16,512))(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # Decoding
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(n_filter*8, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(n_filter*16, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(3, kernel_size=(3, 3), padding="same")(x)
    x = Activation('tanh')(x)
    
    model = Model(inputs=en_img, outputs=x)
    model.summary()
    return (model)
  
def build_discriminator(img_shape):
  
    input_img = Input(shape=img_shape)
    n_filter = 32
    
    x = Conv2D(n_filter*4, kernel_size=(3, 3), padding='same', strides=2)(input_img)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(n_filter*8, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(n_filter*16, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, kernel_size=(3, 3), padding='same', strides=2)(x)
    
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)

    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=out)

    model.summary()
    return model
  
  
def train(encoder, decoder, discriminator, gan, x_train,
          dir_result="./", epochs=10000, batch_size=512):
   
    history = []
    
    if epochs >= 100:
      checkpoint = int(epochs/100)
    elif epochs < 100:
      checkpoint = 1
    
    
    for epoch in range(epochs):
      
        start_epoch = time.time()
        
        np.random.shuffle(x_train)
        
        batches_index = [x for x in range(0, len(x_train)+1, batch_size)]
        
        if batches_index[len(batches_index)-1] != len(x_train):
          batches_index.append(len(x_train))
        #print(batches_index)
        
        for i in range(0, len(batches_index)-1):
          
          start = batches_index[i]
          
          # if i == len(batches_index)-1:
          #   end = len(x_train)
          # else:
          end = batches_index[i+1]
          #print("Batch ",str(i),":",start, end)  
          
          x_batch = x_train[start:end]
          x_batch_size = len(x_batch)
          
          
          # encoder e decoder su immagini reali
          gen_img_enc = encoder.predict(x_batch)
          gen_img_dec = decoder.predict(gen_img_enc)

          noise = get_noise(n_sample=x_batch_size)
          # immagini create dal noise
          fake_img = decoder.predict(noise)

          matrix_true = np.ones((x_batch_size, 1))
          matrix_false = np.zeros((x_batch_size, 1))
          
          # Allenamento discriminatore
          dsc_true = discriminator.train_on_batch(x_batch, matrix_true)
          dsc_true_enc = discriminator.train_on_batch(gen_img_dec, matrix_true)
          dsc_fake = discriminator.train_on_batch(fake_img, matrix_false)

          # Allenamento decoder
          
          noise_gen = get_noise(n_sample=x_batch_size)
          decoder_noise_loss = decoder.train_on_batch(noise_gen, x_batch)
          
          gan_loss = gan.train_on_batch(x_batch, matrix_true)
          
        if epoch % checkpoint == 0 and epoch > 0:
          
          history.append({"Epoch:": epoch,
                          "Discriminator_loss_true:": dsc_true[0],
                          "Discriminator_loss_true_enc:": dsc_true_enc[0],
                          "Discriminator_loss_fake": dsc_fake[0],
                          "Discriminator_acc_fake": dsc_fake[1],
                          "Decoder_noise_loss": decoder_noise_loss,
                          "GAN_Loss:": gan_loss})
          
          print({"Epoch:": epoch,
                          "Discriminator_loss_true:": dsc_true[0],
                          "Discriminator_loss_true_enc:": dsc_true_enc[0],
                          "Discriminator_loss_fake": dsc_fake[0],
                          "Discriminator_acc_fake": dsc_fake[1],
                          "Decoder_noise_loss": decoder_noise_loss,
                          "GAN_Loss:": gan_loss})
          
          noise = get_noise(n_sample=1)

          plot_generated_images(noise, 1, dir_result, epoch)
          
        end_epoch = time.time()
        end_epoch = end_epoch - start_epoch
        print("Ended Epoch {:0.0f}/{:1.0f} in: {:2.4f} s".format(epoch + 1, epochs, end_epoch))

    return history

start = time.time()
   
img_shape = (128, 128, 3)
#Two time-scale update rule
optimizer_std = Adam(0.0001)
optimizer_dsc = Adam(0.0004)

encoder = build_encoder(img_shape)
decoder = build_decoder_gen()
discriminator = build_discriminator(img_shape)

encoder.compile(loss="mse", optimizer=optimizer_std)
decoder.compile(loss="mse", optimizer=optimizer_std)

discriminator.trainable = False

discriminator.compile(loss="mse",
                              optimizer=optimizer_dsc,
                              metrics=["accuracy"])

x = Input(shape=img_shape)
z = encoder(x)
out_dec = decoder(z)
out = discriminator(out_dec)

gan = Model(inputs=x, outputs=out)

gan.compile(loss="mse", optimizer=optimizer_std)

x_train = get_dataset("./img_align_celeba", img_shape)

path = "./drive/My Drive/GAN/res_gan_autoencoder/"

history = train(encoder, decoder, discriminator, gan, x_train,
                  dir_result=path, epochs=100, batch_size=50)

with open(path+"log_gan.txt", "w") as f:
  for item in history:
    item_str = str(item) + "\n"
    f.write(item_str)

# Altrimenti i pesi non verranno salvati.
discriminator.trainable = True

encoder.save(path+"encoder.h5")
decoder.save(path+"decoder.h5")
discriminator.save(path+"discriminator.h5")
# La gan va ricreata a causa di: https://github.com/keras-team/keras/issues/9589
gan.save(path+"gan_autoencoder.h5")

end = time.time()
end = end - start
print("Finished in: " + str(end) + " s")